# IceTop Rigidity Analysis

Before running this notebook, please make sure you have the following .npy files in the appropriate folders:

- energy.npy

- Gweights.npy

- Hweights.npy

- reco_pass.npy

- hits.npy

- particle_type.npy

- showerplane_zen.npy

- laputop_zen.npy

If not, please run save_data.py for the year and interaction model you would like to analyze.

**Note:** This file was made with the ability to compare H4a and GSF. By default, only GSF is used. To enable H4a, go to the code Dictionaries set-up block under the Set-up section. Then go to the SIMS dictionary and uncomment the H4a key.

# Table of Contents
##### 1. [Set-up](#set-up)
##### 2. [Rate vs. Energy](#weighted-rate-vs-energy)
##### 3. [Mean Atomic Charge](#mean-atomic-charge)
##### 4. [Fractional Composition vs. Energy](#fractional-compositions-vs-energy)
##### 5. [Table of fractional comp, ln(A), median log(E), and median log(R)](#table-of-fractional-comp-lna-median-loge-and-median-logR)
##### 6. [Rate vs. Rigidity](#weighted-rate-vs-rigidity)
##### 7. [Energy vs. Rigidity](#energy-vs-rigidity)
##### 8. [Dipole Phase and Amplitude](#dipole-phase-and-amplitude)
##### 9. [Dipole Phase and Amplitude vs. Rigidity and Energy; Rigidity vs. Energy](#dipole-phase-and-amplitude-vs-rigidity-and-energy-rigidity-vs-energy)
##### 10. [Dipole Phase and Amplitude vs. Energy](#dipole-phase-and-amplitude-vs-energy)
##### 11. [Dipole Phase and Amplitude vs. Rigidity](#dipole-phase-and-amplitude-vs-rigidity)
##### 12. [Rigidity vs. Energy](#rigidity-vs-energy)
##### 13. [Opening Angle](#opening-angle)

In [1]:
# Control Panel: Set the number of bins for each graph, the year and the interaction model you want to analyze, and whether to include an "IceCube Preliminary" message
nbins = 50
includePreliminaryMessage = True
'''
Options:
2012
2015
2018
'''
year = 2012
'''
Options (depending on year): 
EPOS-LHC [2012, 2018]
QGSJET-II-04 [2012, 2018]
SIBYLL2.1 [2012, 2015]
SIBYLL2.3 [2012]
SIBYLL2.3d [2018]
'''
model = 'SIBYLL2.1'

## Set-Up

In [2]:
# Import libraries
import os, getpass

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms as mtran

from pandas import DataFrame
from collections import defaultdict

In [3]:
# Load in the SIBYLL2.1 particle sims - note that these will also be your variable names
KEYS = ['energy', 'particle_type', 'hits', 'reco_pass', 'showerplane_zen', 'laputop_zen', 'true_zenith', 'showerplane_az', 'laputop_az', 'true_azimuth', 'Hweights', 'Gweights']

for key in KEYS:
    if os.path.isfile(f'/data/user/tfutrell/it_anisotropy/{year}/{model}/{key}.npy'):
        with open(f'/data/user/tfutrell/it_anisotropy/{year}/{model}/{key}.npy', 'rb') as file:
            globals()[key] = np.load(file)
    else:
        print(f'{key}.npy does not exist or is not in the required folder.')

# Load in the in-ice data
d = np.load("/data/user/juancarlos/notebooks/dst_20904_v3.npz")

In [4]:
# Define a function for weighted medians, or weighted quantiles
def weighted_quantiles(values, weights, quantiles=0.5):
    if len(values) == 0:
        return 0
    else:
        i = np.argsort(values)
        c = np.cumsum(weights[i])
        return values[i[np.searchsorted(c, np.array(quantiles) * c[-1])]]

In [5]:
# Dictionary set-ups
# Set up composition-dependent dictionaries based on year and interaction model (some only have Fe and p)
if year == 2012 and (model == 'EPOS-LHC' or model == 'SIBYLL2.3' or model == 'QGSJET-II-04'):
    # Cuts for different compositions
    COMPS = {
        'proton': particle_type == 2.21200000e+03,
        'iron': particle_type == 1.00026056e+09,
        'true': np.tile(True, len(energy))
    }

    # Graph colors by composition
    COLORS = {
        'true' : 'black',
        'proton': 'red',
        'iron': 'blue'
    }

    # Graph labels by composition
    LABEL = {
        'true' : 'Data',
        'proton': 'p',
        'iron': 'Fe'
    }
else:
    # Cuts for different compositions
    COMPS = {
        'proton': particle_type == 2.21200000e+03,
        'helium' : particle_type == 1.00002004e+09,
        'oxygen': particle_type == 1.00008016e+09,
        'iron': particle_type == 1.00026056e+09,
        'true': np.tile(True, len(energy))
    }

    # Graph colors by composition
    COLORS = {
        'true' : 'black',
        'proton': 'red',
        'helium': 'orange',
        'oxygen': 'purple',
        'iron': 'blue'
    }

    # Graph labels by composition
    LABEL = {
        'true' : 'Data',
        'proton': 'p',
        'helium': 'He',
        'oxygen': 'O',
        'iron': 'Fe'
    }

# amu of different compositions
A = {
    'proton': 1.0073,
    'helium': 4.0026,
    'oxygen': 16,
    'iron': 55.845  
}

# Number of protons for different compositions
Z = {
    'proton': 1,
    'helium': 2,
    'nitrogen': 7,
    'oxygen': 8,
    'aluminum': 13,
    'iron': 26
}

# IceTop Tiers and quality cuts - quality cuts may not work well with some years and Tiers (e.g. reco_succeeded cancels out Tier 4 for 2012)
tier1_quality_cuts = (showerplane_zen < np.radians(55))
quality_cuts = (laputop_zen < np.radians(55)) * (reco_pass == 1)

# Calculation for the Tier offset per year
offset = np.ceil((year - 2011)/2)

if year < 2015:
    TIERS = {
        'Tier 1': (3 <= hits) * (hits < 5),
        'Tier 2': (5 <= hits) * (hits < 10 - offset),
        'Tier 3': (10 - offset <= hits) * (hits < 17 - offset),
        'Tier 4': (17 - offset <= hits)
    }

else:
    TIERS = {
        'Tier 3': (10 - offset <= hits) * (hits < 17 - offset),
        'Tier 4': (17 - offset <= hits)
    }

# Weights
SIMS = {
    'H4a': Hweights,
    'GSF': Gweights
}

# Flux from histogram
FLUX = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))

# Histogram bins
HBINS = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))

# Linestyle by weights
LINESTYLE = {
    'H4a': '--',
    'GSF': None
}

# Dictionary for total flux times log(Z) and for total flux across compositions for each Tier and weight.
# 3-level nested defaultdict: d['key1']['key2']['key3'] = np.zeros(nbins)
TOTALS = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: np.zeros(nbins))))

# Table for partial compositions, ln(A), median log of rigidity and energy for each Tier and weight
TABLE = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))

IC_COMPS = {
    'proton': d['type'] == 2212,
    'helium': d['type'] == 1.00002004e+09,
    'nitrogen': d['type'] == 1.000070140e+09,
    'aluminum': d['type'] == 1.00013027e+09,
    'iron': d['type'] == 1.00026056e+09
}

# To avoid linestyle errors in dipole phase and amplitude graphs
FMT = {
    'H4a': '--o',
    'GSF': '-o'
}

In [ ]:
for tier, tier_cut in TIERS.items():
    if tier == 'Tier 1':
        cuts = tier_cut * (showerplane_zen < np.radians(55))
    else:
        cuts = tier_cut * (reco_pass == 1)
    
    print((weighted_quantiles(energy[cuts], Hweights[cuts])))

In [7]:
# Calculate log10(rigidity)
# The -3 is to scale energy to TeV
rigidity = np.log10(energy) - 3
for comp, comp_cut in COMPS.items():
    if not comp == 'true':
        rigidity[comp_cut] -= np.log10(Z[comp])

ic_rigidity = np.log10(d['mc_energy']) - 3
for comp, comp_cut in IC_COMPS.items():
    if not comp == 'true':
        ic_rigidity[comp_cut] -= np.log10(Z[comp])

## Weighted rate vs. energy

In [7]:
# Prepare the graphs
ncols = len(TIERS)
fig, axs = plt.subplots(figsize=(13 * ncols, 8), ncols=ncols)

for tier, tier_cut, ax in zip(TIERS.keys(), TIERS.values(), axs):
    # Print Tier for table
    print(tier)
    print('--------------------------')
    
    for weight_name, weights in SIMS.items():
        # Print weights for table
        print(weight_name)

        for comp, comp_cut in COMPS.items():
            # Combine cuts
            combined_cut = comp_cut * tier_cut

            if year == 2012:
                if tier == 'Tier 1':
                    combined_cut *= (showerplane_zen < np.radians(55))
                else:
                    combined_cut *= tier1_quality_cuts
            elif year == 2015:
                if tier =='Tier 3':
                    combined_cut *= (laputop_zen < np.radians(55))
                else:
                    combined_cut *= quality_cuts
            else:
                combined_cut *= quality_cuts

            # Set up the histogram arguments
            hist_args = {
                'bins': nbins,
                'linestyle': LINESTYLE[weight_name],
                'log': True,
                'label': f'{LABEL[comp]} ({weight_name})',
                'histtype':'step',
                'color': COLORS[comp]
            }

            # Graph primary energy
            FLUX[comp][tier][weight_name], HBINS[comp][tier][weight_name], patches1 = ax.hist(np.log10(energy[combined_cut]), weights=weights[combined_cut], **hist_args)

            # Print median energy and its first sigma
            if comp == 'proton' or comp == 'iron' or comp == 'true':
                median = weighted_quantiles(energy[combined_cut], weights[combined_cut])
                print(f'    {comp}')
                print(f'        Median: {round(median/1e6, 3)} PeV')
                print(f'        68%: {round((median-weighted_quantiles(energy[combined_cut], weights[combined_cut], quantiles=.16))/1e6, 3)}-{round((weighted_quantiles(energy[combined_cut], weights[combined_cut], .84)-median)/1e6, 3)} PeV')
                print()

    # Label axes and add legend and title
    ax.set_xlabel('$\log_{{10}}$(Energy) (GeV)', size=16)
    ax.set_ylabel('$\log_{{10}}$(Rates) (Hz)', size=16)
    ax.set_title(f'{year} {model} Energy vs. Weighted Rates for {tier}', size=24)
    ax.legend()

## Mean Atomic Charge

In [ ]:
fig, axs = plt.subplots(figsize=(13 * ncols, 8), ncols=ncols)

for tier, ax in zip(TIERS, axs):    
    for weight_name in SIMS:
        for comp in COMPS:
            if not comp == 'true':
                # Sum flux times the number of protons for each comp, Tier and weight
                TOTALS['numerator'][tier][weight_name] += FLUX[comp][tier][weight_name] * np.log10(Z[comp])

                # Sum flux for each comp, Tier and weight
                TOTALS['denominator'][tier][weight_name] += FLUX[comp][tier][weight_name]

        # Calculate <Z>
        meanZ = TOTALS['numerator'][tier][weight_name] / TOTALS['denominator'][tier][weight_name]

        # Plot <Z> vs. energy
        ax.plot(HBINS['true'][tier][weight_name][:-1], meanZ, label=weight_name)

    ax.set_xlabel('$\log_{{10}}$(Energy/GeV)', size=16)
    ax.set_ylabel('Mean $\log_{{10}}(Z)$', size=16)
    ax.set_title(f'{year} {model} Mean $\log_{{10}}(Z)$ vs. Energy for {tier}', size=24)

    ax.legend()

## Fractional Compositions vs. Energy

In [ ]:
# Particle Fraction vs. Energy
fig, axs = plt.subplots(figsize=(13 * ncols, 8), ncols=ncols)

for tier, ax in zip(TIERS, axs):
    for weight_name in SIMS:
        for comp in COMPS:
            # Plot fractional composition
            if not comp == 'true':
                ax.plot(HBINS[comp][tier][weight_name][:-1], FLUX[comp][tier][weight_name] / TOTALS['denominator'][tier][weight_name],
                label=f'{LABEL[comp]} ({weight_name})', linestyle=LINESTYLE[weight_name], color=COLORS[comp])

    ax.set_xlabel('$log_{{10}}$(Energy/GeV)', size=16)
    ax.set_ylabel('Fraction of Particles', size=16)
    ax.set_title(f'{year} {model} Particle Fraction vs. Energy for {tier}', size=24)

    ax.legend()

# Table of fractional comp, ln(A), median log(E), and median log(R)

In [ ]:
print(f'{year} {model}')
print()

for weight_name, weights in SIMS.items():
    # Print weight name for data table
    print(weight_name)
    print()

    for tier, tier_cut in TIERS.items():
        # Start ln_A at zero
        ln_A = 0

        # Set up quality cuts
        combined_cut = tier_cut
        if year == 2012:
            if tier == 'Tier 1':
                combined_cut *= (showerplane_zen < np.radians(55))
            else:
                combined_cut *= (tier1_quality_cuts)
        elif year == 2015:
            if tier =='Tier 3':
                combined_cut *= (laputop_zen < np.radians(55))
            else:
                combined_cut *= quality_cuts
        else:
            combined_cut *= quality_cuts

        for comp, comp_cut in COMPS.items():
            if not comp == 'true':
                # Save the partial compositions to the table
                TABLE[weight_name][tier][f'{comp} part'] = round(np.mean(FLUX[comp][tier][weight_name]) / np.mean(TOTALS['denominator'][tier][weight_name]), 3)

                # Calculate ln(A) with partial compositions
                ln_A += TABLE[weight_name][tier][f'{comp} part'] * np.log(A[comp])

        # Save ln(A) to the table
        TABLE[weight_name][tier]['ln(A)'] = round(ln_A, 3)
        
        # Calculate median log E (Scaled to TeV) and rigidity
        TABLE[weight_name][tier]['median(log(E))'] = round(weighted_quantiles(np.log10(energy[combined_cut]) - 3, weights=weights[combined_cut]), 3)
        TABLE[weight_name][tier]['median(log(R))'] = round(weighted_quantiles(rigidity[combined_cut], weights=weights[combined_cut]), 3)
        median_R = TABLE[weight_name][tier]['median(log(R))']

    # Print out the table for that weight
    print(DataFrame(TABLE[weight_name]))
    print()

## Weighted rate vs rigidity

In [ ]:
fig, axs = plt.subplots(figsize=(13 * ncols, 8), ncols=ncols)

for tier, tier_cut, ax in zip(TIERS.keys(), TIERS.values(), axs):
    for weight_name, weights in SIMS.items():
        for comp, comp_cut in COMPS.items():
            # Set up the histogram arguments
            hist_args = {
                'bins': nbins,
                'linestyle': LINESTYLE[weight_name],
                'log': True,
                'label': f'{LABEL[comp]} ({weight_name})',
                'histtype':'step',
                'color': COLORS[comp]
            }

            # Graph rate vs. rigidity
            combined_cut = tier_cut * comp_cut

            if year == 2012:
                if tier == 'Tier 1':
                    combined_cut *= (showerplane_zen < np.radians(55))
                else:
                    combined_cut *= (tier1_quality_cuts)
            elif year == 2015:
                if tier =='Tier 3':
                    combined_cut *= (laputop_zen < np.radians(55))
                else:
                    combined_cut *= quality_cuts
            else:
                combined_cut *= quality_cuts

            ax.hist(rigidity[combined_cut], weights=weights[combined_cut], **hist_args)
            #ax.scatter(weighted_quantiles(rigidity[combined_cut], weights=weights[combined_cut]), np.median(rate[0]), label=f'Median {comp} ({weight_name})', c=COLORS[comp])

    ax.set_xlabel('$\log_{{10}}$(Rigidity) (TV)', size=16)
    ax.set_ylabel('$\log_{{10}}$(Rate) (Hz)', size=16)
    ax.set_title(f'{year} {model} Rate vs. Rigidity for {tier}', size=24)
    ax.legend()

## Energy vs. rigidity

In [ ]:
fig, axs = plt.subplots(figsize=(13, 8))

for tier_cut in TIERS.values():
    for comp, comp_cut in COMPS.items():
        if not comp == 'true':
            combined_cut = comp_cut * tier_cut

            if year == 2012:
                if tier == 'Tier 1':
                    combined_cut *= (showerplane_zen < np.radians(55))
                else:
                    combined_cut *= (tier1_quality_cuts)
            elif year == 2015:
                if tier =='Tier 3':
                    combined_cut *= (laputop_zen < np.radians(55))
                else:
                    combined_cut *= quality_cuts
            else:
                combined_cut *= quality_cuts

            # Graph energy vs. rigidity
            axs.plot(np.log10(energy[combined_cut]), rigidity[combined_cut], '.-', label=LABEL[comp], color=COLORS[comp])
    
    axs.set_xlabel('$\log_{{10}}$(Energy) (TV)', size=16)
    axs.set_ylabel('$\log_{{10}}$(Rigidity) (GeV)', size=16)
    axs.set_title(f'{year} {model} Rigidity vs. Energy', size=24)

    axs.legend()
plt.show()

# Dipole Phase and Amplitude vs. Energy and Rigidity; Rigidity vs. Energy

In [14]:
# Set-up
# Define label parameters
lParams = {'size':'xx-large','color':'black','family':'sans-serif'}

# Set up in-ice energy bins
BINS = {
'4.125': (d['reco_energy'] > 4) * (d['reco_energy'] < 4.25),
'4.375': (d['reco_energy'] > 4.25) * (d['reco_energy'] < 4.5),
'4.625': (d['reco_energy'] > 4.5) * (d['reco_energy'] < 4.75),
'4.875': (d['reco_energy'] > 4.75) * (d['reco_energy'] < 5),
'5.125': (d['reco_energy'] > 5) * (d['reco_energy'] < 5.25),
'5.375': (d['reco_energy'] > 5.25) * (d['reco_energy'] < 5.5),
'5.75': (d['reco_energy'] > 5.5) * (d['reco_energy'] < 6),
'6.25': (d['reco_energy'] > 6) * (d['reco_energy'] < 6.5), 
'7.25': (d['reco_energy'] > 6.5) * (d['reco_energy'] < 8)
}

if year < 2015:
    it_amplitude = 1.74 * np.array([4.9, 13.5, 11.9, 12.5])
    it_phase = [-96.1, -93.8, -72.2, -64.1]

    it_amplitude_error = [1.1, 1.2, 1.7, 4.2]
    it_phase_err = [12.6, 5.0, 7.4, 16.1]
else:
    it_amplitude = 1.74 * np.array([11.9, 12.5])
    it_phase = [-72.2, -64.1]

    it_amplitude_error = [1.7, 4.2]
    it_phase_err = [7.4, 16.1]

# In-ice amplitude (with upper limit), phase, and associated errors
ic_amplitude = [13.33, 11.24, 8.28, 4.81, 2.04, 3.6, 7.2, 14.0, 18.0]
ic_upper_limit = [13.33, 11.24, 8.28, 4.81, 2.04, 3.6, 7.2, 14.0, 33.4]
ic_phase = [52.76, 50.91, 49.2, 41.05, 18.0, -71.0, -86.0, -85.0, -107.0]

ic_amplitude_error = [0.05, 0.05, 0.09, 0.17, 0.33, 0.5, 0.8, 2.4, 7]
ic_phase_err = [0.20, 0.28, 0.6, 2.0, 9, 9, 6, 10, 22]

# Calculate in-ice rigidity
ic_rigidity = np.log10(d['mc_energy']) - 3
for comp, comp_cut in IC_COMPS.items():
    ic_rigidity[comp_cut] -= np.log10(Z[comp])

# In-ice quality cuts
qual_cut = (d['zenith'] > 0.002) * (d['zenith'] < (np.pi - 0.002)) * (d['nchannel'] >= 10) * (d['mc_energy'] == d['mc_energy']) * (d['rlogl'] < 25)

# Calculate median energies and rigidities for IceTop and IceCube
IT_MEDIAN_E = {
    'H4a': [np.log10(weighted_quantiles(energy[tier_cut], Hweights[tier_cut])) - 3 for tier_cut in TIERS.values()],
    'GSF': [np.log10(weighted_quantiles(energy[tier_cut], Gweights[tier_cut])) - 3 for tier_cut in TIERS.values()]
}

IC_MEDIAN_E = {
    'H4a': [np.log10(weighted_quantiles(d['mc_energy'][bin_cut], d['GaisserH4a'][bin_cut])) - 3 for bin_cut in BINS.values()],
    'GSF': [np.log10(weighted_quantiles(d['mc_energy'][bin_cut], d['GlobalSplineFit5Comp'][bin_cut])) - 3 for bin_cut in BINS.values()]
}

IC_MEDIAN_R = {
    'H4a': [weighted_quantiles(ic_rigidity[bin_cut * qual_cut], d['GaisserH4a'][bin_cut * qual_cut]) for bin_cut in BINS.values()],
    'GSF': [weighted_quantiles(ic_rigidity[bin_cut * qual_cut], d['GlobalSplineFit5Comp'][bin_cut * qual_cut]) for bin_cut in BINS.values()]
}

IT_MEDIAN_R = {
    'H4a': [],
    'GSF': []
}

for weight_name, weights in SIMS.items():
    for tier, tier_cut in TIERS.items():
        if tier == 'Tier 1':
            IT_MEDIAN_R[weight_name].append(weighted_quantiles(rigidity[tier_cut * (showerplane_zen < np.radians(55))], weights[tier_cut * (showerplane_zen < np.radians(55))]))
        else:
            IT_MEDIAN_R[weight_name].append(weighted_quantiles(rigidity[tier_cut * quality_cuts], weights[tier_cut * quality_cuts]))

## Dipole phase and amplitude vs. energy

In [ ]:
fig, axs = plt.subplots(figsize=(26, 8), ncols=2)

for weight_name in SIMS:
    # Plot amplitude vs. energy
    axs[0].set_yscale("log", nonpositive='mask')

    axs[0].plot(IC_MEDIAN_E[weight_name][-2:], ic_upper_limit[-2:], '-o', label=f'In-ice {weight_name} upper limit', c='purple')
    axs[0].errorbar(IC_MEDIAN_E[weight_name], ic_amplitude, yerr=ic_amplitude_error, label=f'In-ice {weight_name}', c='blue', fmt=FMT[weight_name])
    axs[0].errorbar(IT_MEDIAN_E[weight_name], it_amplitude, yerr=it_amplitude_error, label=f'IceTop {weight_name}', c='orange', fmt=FMT[weight_name])

    axs[0].set_xlabel('$\log_{{10}}$(True Energy/TeV)', size=16)
    axs[0].set_ylabel('Amplitude ($10^{{-4}}$)', size=16)
    axs[0].set_title(f'{year} {model} Dipole Amplitude vs. True Energy', size=24)
    if includePreliminaryMessage:
        axs[0].annotate('IceCube Preliminary', xy=(3.0, 5), **lParams)
    
    axs[0].grid(visible=True)
    axs[0].legend()

    # Plot phase vs. energy
    axs[1].errorbar(IC_MEDIAN_E[weight_name], ic_phase, yerr=ic_phase_err, label=f'In-ice {weight_name}', c='blue', fmt=FMT[weight_name])
    axs[1].errorbar(IT_MEDIAN_E[weight_name], it_phase, yerr=it_phase_err, label=f'I {weight_name}', c='orange', fmt=FMT[weight_name])

    axs[1].set_xlabel('$\log_{{10}}$(True Energy/TeV)', size=16)
    axs[1].set_ylabel('Phase ($\degree$)', size=16)
    axs[1].set_title(f'{year} {model} Dipole Phase vs. True Energy', size=24)
    if includePreliminaryMessage:
        axs[1].annotate('IceCube Preliminary', xy=(1.25, -65), **lParams)

    axs[1].grid(visible=True)
    axs[1].legend()
    
plt.show()

## Dipole phase and amplitude vs. rigidity

In [ ]:
fig, axs = plt.subplots(figsize=(26, 8), ncols=2)

for weight_name in SIMS:
    # Plot amplitude vs. rigidity
    axs[0].set_yscale("log", nonpositive='mask')

    axs[0].plot(IC_MEDIAN_R[weight_name][-2:], ic_upper_limit[-2:], FMT[weight_name], label=f'In-ice {weight_name} upper limit', c='purple')
    axs[0].errorbar(IC_MEDIAN_R[weight_name], ic_amplitude, yerr=ic_amplitude_error, label=f'In-ice {weight_name}', c='blue', fmt=FMT[weight_name])
    axs[0].errorbar(IT_MEDIAN_R[weight_name], it_amplitude, yerr=it_amplitude_error, label=f'IceTop {weight_name}', c='orange', fmt=FMT[weight_name])

    # Plot phase vs. rigidity
    axs[1].errorbar(IC_MEDIAN_R[weight_name], ic_phase, yerr=ic_phase_err, label=f'In-ice {weight_name}', c='blue', fmt=FMT[weight_name])
    axs[1].errorbar(IT_MEDIAN_R[weight_name], it_phase, yerr=it_phase_err, label=f'IceTop {weight_name}', c='orange', fmt=FMT[weight_name])
    
axs[0].set_xlabel('$\log_{{10}}$(True Rigidity/TV)', size=16)
axs[0].set_ylabel('Amplitude ($10^{{-4}}$)', size=16)
axs[0].set_title(f'{year} {model} Dipole Amplitude vs. True Rigidity', size=24)
if includePreliminaryMessage:
    axs[0].annotate('IceCube Preliminary', xy=(2.5, 5), **lParams)

axs[0].grid(visible=True)
axs[0].legend()

axs[1].set_xlabel('$\log_{{10}}$(True Rigidity/TV)', size=16)
axs[1].set_ylabel('Phase ($\degree$)', size=16)
axs[1].set_title(f'{year} {model} Dipole Phase vs. True Rigidity', size=24)
if includePreliminaryMessage:
    axs[1].annotate('IceCube Preliminary', xy=(1.15,-65), **lParams)
 
axs[1].grid(visible=True)
axs[1].legend()
    
plt.show()

## Rigidity vs. energy

In [ ]:
fig, axs = plt.subplots(figsize=(13, 8))

for weight_name, weights in SIMS.items():
    ic_weights = []
    if weight_name == 'H4a':
        ic_weights = d['GaisserH4a']
    elif weight_name == 'GSF':
        ic_weights = d['GlobalSplineFit5Comp']

    # Plot energy vs. rigidty
    ic_energy_err = ([], [])
    ic_rigidity_err = ([], [])
    for bin_cut, median_e, median_r in zip(BINS.values(), IC_MEDIAN_E[weight_name], IC_MEDIAN_R[weight_name]):
        combined_cut = bin_cut * qual_cut

        ic_energy_err[0].append(abs(np.log10(weighted_quantiles(d['mc_energy'][combined_cut], ic_weights[combined_cut], quantiles=.16)) - 3 - median_e))
        ic_energy_err[1].append(abs(np.log10(weighted_quantiles(d['mc_energy'][combined_cut], ic_weights[combined_cut], quantiles=.84)) - 3 - median_e))
        ic_rigidity_err[0].append(abs(weighted_quantiles(ic_rigidity[combined_cut], ic_weights[combined_cut], quantiles=.16) - median_r))
        ic_rigidity_err[1].append(abs(weighted_quantiles(ic_rigidity[combined_cut], ic_weights[combined_cut], quantiles=.84) - median_r))

    it_energy_err = ([], [])
    it_rigidity_err = ([], [])
    for tier, tier_cut, median_e, median_r in zip(TIERS.keys(), TIERS.values(), IT_MEDIAN_E[weight_name], IT_MEDIAN_R[weight_name]):
        combined_cut = None
        if tier == 'Tier 1':
            combined_cut = tier_cut * (showerplane_zen < np.radians(55))
        else:
            combined_cut = tier_cut * quality_cuts

        it_energy_err[0].append(abs(np.log10(weighted_quantiles(energy[combined_cut], weights[combined_cut], quantiles=.16)) - 3 - median_e))
        it_energy_err[1].append(abs(np.log10(weighted_quantiles(energy[combined_cut], weights[combined_cut], quantiles=.84)) - 3 - median_e))
        it_rigidity_err[0].append(abs(weighted_quantiles(rigidity[combined_cut], weights[combined_cut], quantiles=.16) - median_r))
        it_rigidity_err[1].append(abs(weighted_quantiles(rigidity[combined_cut], weights[combined_cut], quantiles=.84) - median_r))

    plt.errorbar(IC_MEDIAN_E[weight_name], IC_MEDIAN_R[weight_name], xerr=ic_energy_err, yerr=ic_rigidity_err, fmt=FMT[weight_name], label=f'In-ice {weight_name}', color='blue')
    plt.errorbar(IT_MEDIAN_E[weight_name], IT_MEDIAN_R[weight_name], xerr=it_energy_err, yerr=it_rigidity_err, fmt=FMT[weight_name], label=f'IceTop {weight_name}', color='orange')

plt.xlabel('$\log_{{10}}$(True Energy/TeV)', size=16)
plt.ylabel('$\log_{{10}}$(True Rigidiy/TV)', size=16)
plt.title(f'{year} {model} In-ice and IceTop Rigidity vs. Energy', size=24)
if includePreliminaryMessage:
    plt.annotate('IceCube Preliminary', xy=(3.25, 1.25), **lParams)

plt.grid(visible=True, which='both')
plt.legend()
    
plt.show()

# Opening Angle

In [ ]:
ncols = len(TIERS.keys())
fig, axs = plt.subplots(figsize=(13 * ncols, 8), ncols=ncols)

title_params = {'fontsize':24}
label_params = {'fontsize':16}

npoints = 11

coszbins = np.linspace(1, np.cos(65 * np.pi / 180), npoints + 1)
zbins = np.arccos(coszbins)
zmids = (zbins[1:]+zbins[:-1]) / 2

for ax, tier, tier_cut in zip(axs, TIERS.keys(), TIERS.values()):
    print(tier)
    print('--------------')
    combined_cut = tier_cut

    reco_zen = np.array([])
    reco_az = np.array([])

    if tier == 'Tier 1':
        reco_zenith = showerplane_zen
        reco_az = showerplane_az
    else:
        reco_zenith = laputop_zen
        reco_az = laputop_az
        combined_cut *= (reco_pass == 1)

    for comp, comp_cut in COMPS.items():
        if comp == 'proton' or comp == 'iron':
            print(f'    {comp}')
            scatter_args = {'label': f'{LABEL[comp]} ({weight_name})', 'color': COLORS[comp]}
            plot_cut = combined_cut * comp_cut

            zbin_values = np.digitize(reco_zenith, zbins) - 1

            def delta_omega (reco_zen, reco_azi, true_zen, true_azi):
                return np.arccos(np.sin(reco_zen) * np.sin(true_zen) + np.cos(reco_zen) * np.cos(true_zen) * np.cos(reco_azi - true_azi))

            median_o = weighted_quantiles(delta_omega(0.5 * np.pi - reco_zenith[plot_cut * (reco_zenith < np.radians(55))], reco_az[plot_cut * (reco_zenith < np.radians(55))], 0.5 * np.pi - true_zenith[plot_cut * (reco_zenith < np.radians(55))], true_azimuth[plot_cut * (reco_zenith < np.radians(55))]), Gweights[plot_cut * (reco_zenith < np.radians(55))])
            print(f'        {round(180 / np.pi * median_o, 3)}')
            print()

            x = 180 / np.pi * zmids
            y = [180 / np.pi * weighted_quantiles(delta_omega(0.5 * np.pi - reco_zenith[plot_cut * (zbin_values == i)], reco_az[plot_cut * (zbin_values == i)], 0.5 * np.pi - true_zenith[plot_cut * (zbin_values == i)], true_azimuth[plot_cut * (zbin_values == i)]), Gweights[plot_cut * (zbin_values == i)]) for i in range(0, npoints)]
            
            x_err = [90 / np.pi * (zbins[i + 1] - zbins[i]) for i in range(0, npoints)]
            y_err = ([180 / np.pi * weighted_quantiles(delta_omega(0.5 * np.pi - reco_zenith[plot_cut * (zbin_values == i)], reco_az[plot_cut * (zbin_values == i)], 0.5 * np.pi - true_zenith[plot_cut * (zbin_values == i)], true_azimuth[plot_cut * (zbin_values == i)]), weights=Gweights[plot_cut * (zbin_values == i)], quantiles=.16) for i in range(0, npoints)], [180 / np.pi * weighted_quantiles(delta_omega(0.5 * np.pi - reco_zenith[plot_cut * (zbin_values == i)], reco_az[plot_cut * (zbin_values == i)], 0.5 * np.pi - true_zenith[plot_cut * (zbin_values == i)], true_azimuth[plot_cut * (zbin_values == i)]), weights=Gweights[plot_cut * (zbin_values == i)], quantiles=.84) for i in range(0, npoints)])
            
            ax.errorbar(x, y, xerr=x_err, yerr=y_err, fmt='o', **scatter_args)

    ax.set_xlim((0, 68))
    ax.set_xticks(np.linspace(0, 68, 18))
    ax.axvline(x=55, color="black", linestyle="--")
    

    ax.set_yscale("log", nonpositive='mask')
    ax.set_ylim((0.1, 100))

    ax.set_title(f'{tier} Direction resolution ({year} {model})', **title_params)
    ax.set_xlabel('θº', **label_params)
    ax.set_ylabel('ΔΩº', **label_params)
    ax.legend()

plt.show()